In [115]:

!pip install --pre pycaret
!pip install pycaret


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [116]:
from __future__ import absolute_import, division, print_function, unicode_literals
from google.colab import files
import pandas as pd
import io
import numpy as np
import os

from datetime import datetime
#from download import download

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from google.colab import files
import time
import numpy as np
import pandas as pd
#import numpy as np
from pycaret.datasets import get_data
from pycaret.time_series import TSForecastingExperiment
from statsmodels.tsa.stattools import adfuller
import plotly.express as px
from statsmodels.tsa.stattools import kpss
mpl.rcParams['figure.figsize']=(8,6)
mpl.rcParams['axes.grid']=False

In [117]:
#NECESSARY IMPORTS
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from statsmodels.tsa.api import VAR, adfuller, kpss
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import pandas as pd
import numpy as np
from keras.optimizers import RMSprop
from tensorflow.keras import Sequential, Model
from tensorflow import keras
from tensorflow.keras.metrics import RootMeanSquaredError , MeanAbsoluteError
from keras.layers import UpSampling1D,Input, Dense, SimpleRNN , Dropout,DepthwiseConv1D,Reshape, Attention, SpatialDropout1D,LSTM,LocallyConnected1D, GlobalMaxPooling1D,AveragePooling1D, Bidirectional, TimeDistributed, GRU, LayerNormalization, ConvLSTM1D
from keras.layers.convolutional import MaxPooling1D, Conv1D
from keras.layers import Flatten, ConvLSTM2D
from keras.regularizers import l1, l2
from keras.layers import RepeatVector
import time
import random
#!pip install pandasql
import tensorflow as tf
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
random.seed(1337)
np.random.seed(1337)
tf.random.set_seed(1337)
#import pandasql as ps
import matplotlib as mpl
import matplotlib.pyplot as plt
#from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals()) 
from sklearn.metrics import mean_squared_error

In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
##########################################################
################ Data Preprocessing ######################
##########################################################

# Below function deals with converting the data into time series data with specific number of lags/window/timesteps

def one_step_forecast(df, window):
    d = df.values
    x = []
    n = len(df)
    idx = df.index[:-window]
    for start in range(n-window):
        end = start + window
        x.append(d[start:end])
    cols = [f'x_{i}' for i in range(1, window+1)]
    x = np.array(x).reshape(n-window, -1)
    y = df.iloc[window:].values
    df_xs = pd.DataFrame(x, columns=cols, index=idx)
    df_y = pd.DataFrame(y.reshape(-1), columns=['y'], index=idx)
    return pd.concat([df_xs, df_y], axis=1).dropna()

# Below class deals with the Z-score Normalization / Splitting of the DataSet
# Here Ratio of Test , Train and Split is 81:9:10
# It can be reset using changing the split variable in __init__ function

class Standardize:
  def __init__(self, df, split=0.20):
    self.data=df
    self.split=split
  def split_data(self):
    n=int(len(self.data)*self.split)
    train, test=self.data.iloc[:-n], self.data.iloc[-n:]
    n=int(len(train)*self.split)
    train, val=train.iloc[:-n], train.iloc[-n:]
    assert len(test)+len(train)+len(val)==len(self.data)
    return train, test, val
    '''
  def split_data_view(self):
    n=int(len(self.data)*self.split)
    train, test=self.data.iloc[:-n], self.data.iloc[-n:]
    n=int(len(train)*self.split)
    train, val=train.iloc[:-n], train.iloc[-n:]
    assert len(test)+len(train)+len(val)==len(self.data)
    return train, val
    '''
  def transform(self, data):
    data_s=(data-self.mu)/self.sigma
    return data_s
  def fit_transform(self):
    train, test, val=self.split_data()
    self.mu, self.sigma= train.mean(),train.std()
    train_s=self.transform(train)
    test_s=self.transform(test)
    val_s=self.transform(val)
    return train_s, test_s, val_s
    '''
  def fit_transform_view(self):
    train, test, val=self.split_data_view()
    self.mu, self.sigma= train.mean(),train.std()
    train_s=self.transform(train)
    test_s=self.transform(test)
    val_s=self.transform(val)
    return train_s, test_s, val_s
    '''
  def inverse(self, data):
    return (data*self.sigma)+self.mu
  def inverse_y(self, data):
    return (data*self.sigma[-1])+self.mu[-1]


# This function deals with getting out the X and Y out of the dataset
# Column earlier named as 'y' will be taken out for processing, rest will be kept as x

def features_target_ts(*args):
        y=[col.pop('y').values.reshape(-1,1) for col in args]
        #yk=[col.pop('var5(t)').values.reshape(-1,1) for col in args]
        x=[col.values.reshape(*col.shape,1) for col in args]
        return *y, *x



#####################################################
################# Model Creation ####################
#####################################################
def create_model(train, units, dropout=0.2):
        # Variable-length int sequences.

        
        model_LSTM = Sequential()  
        model_LSTM.add(LSTM(256,activation='relu',input_shape=(train.shape[1], train.shape[2])))
        model_LSTM.add(Dense(1))
        model_LSTM._name='LSTM'
                  
        model_GRU = Sequential()  
        model_GRU.add(GRU(256,activation='relu',input_shape=(train.shape[1], train.shape[2])))
        model_GRU.add(Dense(1))
        model_GRU._name='GRU'
                      
        model_RNN = Sequential()  
        model_RNN.add(SimpleRNN(256,activation='relu',input_shape=(train.shape[1], train.shape[2])))
        model_RNN.add(Dense(1))
        model_RNN._name='RNN'
                      
        model_BiLSTM = Sequential()  
        #model_BiLSTM.add(Bidirectional(LSTM(256,activation='relu',input_shape=(train.shape[1], train.shape[2]))))
        model_BiLSTM.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(train.shape[1], train.shape[2])))
        #print(model_BiLSTM.output_shape)
        model_BiLSTM.add(Flatten())
        #print(model_BiLSTM.output_shape)
        model_BiLSTM.add(Dense(1))
        #print(model_BiLSTM.output_shape)
        model_BiLSTM._name='BiLSTM'
        #model_BiLSTM.add(Dense(1))
                  
        model_CNN = Sequential()  
        model_CNN.add(Conv1D(filters=256, kernel_size=1,input_shape=(train.shape[1], train.shape[2])))
        model_CNN.add(Flatten())
        model_CNN.add(Dense(1))
        model_CNN._name='CNN'
        return  model_LSTM, model_GRU, model_CNN, model_RNN, model_BiLSTM

# This function is responsible for model compilation and training 
def train_model_ts(model, x_train, y_train, x_val, y_val, epochs=250, patience =15, batch_size=66):
          model.compile(optimizer='adam', loss='mean_squared_error', metrics=[RootMeanSquaredError(), MeanAbsoluteError()])
          es=keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=patience)
          history=model.fit(x_train, y_train, shuffle=False, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val),callbacks=[es],verbose=1)
          return history

# This function is responsible for Loss and Validation Loss Graph along with Test Prediction
def plot_forecast(model, x_test, y_test, index, history):
          #fig, ax=plt.subplots(2,1)
          #(pd.Series(history.history['loss']).plot(style='k', alpha=0.50, title='Loss By Epoch',ax=ax[0], label='loss'))
          #(pd.Series(history.history['val_loss']).plot(style='k', alpha=0.50, title='Loss By Epoch',ax=ax[0], label='val_loss'))
          #ax[0].legend()
          predicted=model.predict(x_test)
          #pd.Series(y_test.reshape(-1), index=index).plot(style='k--', alpha=0.5, ax=ax[1], title='Forecast Vs Actual', label='actual')
          #pd.Series(predicted.reshape(-1), index=index).plot(style='k', ax=ax[1],  label='Forecast')
          #fig.tight_layout()
          #ax[1].legend()
          #plt.show()
          return predicted

def Measures(y_true,y_pred):
  '''All input in this function must be 'list' data structure'''

  from sklearn.metrics import mean_squared_error
  from sklearn.metrics import mean_absolute_percentage_error
  from sklearn.metrics import mean_absolute_error
  import math

  a=np.array(y_true)
  b=np.array(y_pred)
  rmse=math.sqrt(sum(pow(a-b,2))/len(a))
  mape=sum(abs((a-b)/b))/len(a)*100

  #print('rmse:',rmse)
  #print('mape:',mape)
  return([rmse,mape])

def DataTrainTestVal(df_new):   
    en_df=one_step_forecast(df_new, 12)
    #en_df

    # Object of Standardize class has been created with dataset as the parameter
    scale_en=Standardize(en_df)
    train_en, test_en, val_en=scale_en.fit_transform()
    
    # Spliting into test, train and validation set
    y_train_en, y_val_en, y_test_in, x_train_en, x_val_en, x_test_en= features_target_ts(train_en, val_en, test_en)
    return (y_train_en, y_val_en, y_test_in, x_train_en, x_val_en, x_test_en, test_en, scale_en, en_df, val_en, train_en) 


#execution of the models here
def Model_Execution(models,x_train_en, y_train_en, x_val_en, y_val_en, x_test_en, y_test_in, test_en, scale_en, en_df, val_en, train_en, batch_size=64):
  view_accuracy=[]
  Test_Predictions=pd.DataFrame()
  Train_Predictions=pd.DataFrame()
  for i in models:
              #i.summary()
              print(i._name)
              history_en_proposed=train_model_ts(i, x_train_en, y_train_en, x_val_en, y_val_en,  batch_size=64)
              Test_len=len(y_test_in)
              print("Model Trained !!! ")

              test_predicted=plot_forecast(i, x_test_en, y_test_in, test_en.index, history_en_proposed)

              scaled_test_y=scale_en.inverse_y(y_test_in)
              #print("Model Predicted For Test Data!!! ")
              #print('len(x_test_en)',len(scaled_test_y))
              #print('x_test_en values:\n',scaled_test_y)

              
              scaled_Pred=scale_en.inverse_y(test_predicted)
              Act_Pred=scaled_Pred
              scaled_Pred=pd.DataFrame(scaled_Pred)
              ll=en_df.iloc[-Test_len:,-1:]

              RMSE_New=(mean_squared_error(scaled_Pred, ll)**0.5)
              #print('Data views number ===>',serial,'<========','RMSE_New:',RMSE_New)
              view_accuracy.append(RMSE_New)
              
              scaled_Pred=scaled_Pred.rename(columns={0:'hb'})
              ll['pd']=scaled_Pred['hb'].values
              ll.plot()

              val_prediction=plot_forecast(i, x_val_en, y_val_en, val_en.index, history_en_proposed)
              print("Validation Data Predicted !!! ")

              Training_prediction=plot_forecast(i, x_train_en, y_train_en, train_en.index, history_en_proposed)
              print("Training Data Predicted !!! ")

              Total_Train_Prediction=np.append(Training_prediction, val_prediction)
              scaled_Train_Pred=scale_en.inverse_y(Total_Train_Prediction)
              Act_Train_Pred=scaled_Train_Pred

              print("Model To be Executed :")
              print(i)
              if i._name=='Proposed':
                  Test_Predictions['Proposed']=pd.DataFrame(Act_Pred)
                  Train_Predictions['Proposed']=pd.DataFrame(Act_Train_Pred)
              if i._name=='LSTM':
                  Test_Predictions['LSTM']=pd.DataFrame(Act_Pred)
                  Train_Predictions['LSTM']=pd.DataFrame(Act_Train_Pred)  
              if i._name=='GRU':
                  Test_Predictions['GRU']=pd.DataFrame(Act_Pred)
                  Train_Predictions['GRU']=pd.DataFrame(Act_Train_Pred)
              if i._name=='CNN':
                  Test_Predictions['CNN']=pd.DataFrame(Act_Pred)
                  Train_Predictions['CNN']=pd.DataFrame(Act_Train_Pred)
              if i._name=='RNN':
                  Test_Predictions['RNN']=pd.DataFrame(Act_Pred)
                  Train_Predictions['RNN']=pd.DataFrame(Act_Train_Pred)
              if i._name=='BiLSTM':
                  Test_Predictions['BiLSTM']=pd.DataFrame(Act_Pred)
                  Train_Predictions['BiLSTM']=pd.DataFrame(Act_Train_Pred)
  return(view_accuracy,Test_Predictions.values.tolist(),scaled_test_y)


#Finding the minimum seasonality of the dataset
def seasonality_min_val(df):
    fh = 50 # or alternately fh = np.arange(1,13)
    fold = 5

    fig_kwargs={'renderer': 'notebook'}
    eda = TSForecastingExperiment()

    dd=eda.setup(data=df, fh=fh, fig_kwargs=fig_kwargs)

    k=pd.DataFrame(dd.significant_sps)

    seasonality=k.to_numpy().flatten()
    seasonality_min=np.amin(seasonality)
    #print("Min seasonality_min:", seasonality_min)
    return(seasonality_min)



#execution of traditional models of deep learning for time series analysis
def Traditional_TS_Model(df):
    Traditional_model_pred=[]
    y_train_en, y_val_en, y_test_in, x_train_en, x_val_en, x_test_en, test_en, scale_en, en_df, val_en, train_en =DataTrainTestVal(df)
    models=[]
    #view_predictions.append(x_test_en[:-1].tolist())
    model_LSTM, model_GRU, model_CNN, model_RNN, model_BiLSTM =create_model(x_train_en, units=32)

    #models.append(model_BiLSTM)
    #models.append(model_LSTM)
    #models.append(model_GRU)
    models.append(model_CNN)
    #models.append(model_RNN)

    view_accuracy,Test_Predictions,y_test_original=Model_Execution(models,x_train_en, y_train_en, x_val_en, y_val_en, x_test_en, y_test_in, test_en, scale_en, en_df, val_en, train_en, batch_size=64)
    #view_accuracy.append(view_accuracy)
    
    #print()
    Traditional_model_pred.append(Test_Predictions)
    return(Traditional_model_pred,train_en, val_en, models, x_test_en, y_test_in, test_en,y_test_original)

#
def Multiview_Data_Preparation(df,view_max):
  data_array=df.to_numpy()
  data_length=len(data_array)
  Train_Data_Len=data_length*0.8
  seasonality_min=seasonality_min_val(df)  #obtaining the minimum seasonal period of the dataset

  multiply=round((Train_Data_Len/view_max)/seasonality_min)
  ViewsDataset=[]
  seasonal=[seasonality_min]
  k=0
  for j in seasonal:
    print(len(data_array[-(j*multiply):-1].tolist()))
    ViewsDataset.append(data_array[-(j*multiply):-1].tolist())
    for i in range(1,view_max):
      print(len(data_array[-(j*multiply)*(i+1):-(j*multiply)*(i)].tolist()))
      ViewsDataset.append(data_array[-(j*multiply)*(i+1):-(j*multiply)*(i)].tolist())
    k=k+1
  return(ViewsDataset)


In [120]:
def MVL_results_compile(Traditional_model_pred, view_predictions, y_test_original):
    #traditional model prediction value adjustment
    model_pred=Traditional_model_pred[0][:][:]
    Trad_model_pred=[]
    for k in model_pred:
      Trad_model_pred.append(k[0])

    #Multi-view ensemble of the predictions value adjustment

    views_model_pred=[]
    for l in range(len(view_predictions)):
      views_pred=view_predictions[l][:][:]
      views_model_pred_i=[]
      for k in views_pred:
        views_model_pred_i.append(k[0])
      views_model_pred.append(views_model_pred_i)

    p=np.transpose(np.array(views_model_pred))
    mean_val_MVL=np.mean(p, axis=1)
    MVL_pred=mean_val_MVL
    #------------------------------------

    #converting y_true original in the list formate
    y_true=[]
    for i in range(len(y_test_original)):
      y_true.append(y_test_original[i][0].tolist())

    #checking the performances

    performance_model=Measures(y_true,Trad_model_pred)
    performance_MVL=Measures(y_true,MVL_pred)

    #print("performance_model:",performance_model)
    #print("performance_MVL:",performance_MVL)

    return([performance_model[0],performance_MVL[0],performance_model[1],performance_MVL[1]])

In [121]:
#df= pd.read_csv('/content/drive/MyDrive/AI LAB CoLab/GradientBias time series analysis/Time Series Dataset/TS_datasets_processed/SunSpotMonthly_Data.csv')
#df= pd.read_csv('/content/drive/MyDrive/AI LAB CoLab/GradientBias time series analysis/Time Series Dataset/TS_datasets_processed/Daily_Sunspot_data.csv')

df= pd.read_csv('/content/drive/MyDrive/AI LAB CoLab/GradientBias time series analysis/Time Series Dataset/TS_datasets_processed/POWER_Point_Daily_Temperature_Data.csv')


In [ ]:
Results_compile=[]
results_view_acc=[]
#generating multiple views of the dataset based on number of views
Views_upto=10


for view_max in range(2,Views_upto+1):
    ViewsDataset=Multiview_Data_Preparation(df,view_max)

    #executing the traditional models here....
    view_accuracy=[]
    df.columns = ['y']

    Traditional_model_pred, train_en, val_en, models, x_test_en, y_test_in, test_en,y_test_original=Traditional_TS_Model(df)

    #multi-view time series analysis code here.....
    serial=0
    view_predictions=[]

    for view in ViewsDataset:
      print('length of the views:',len(view))
      df_new = pd.DataFrame(view)
      df_new.columns = ['y']
      y_train_en, y_val_en, y_test_in_na, x_train_en, x_val_en, x_test_en_na, test_en_na, scale_en, en_df, val_en_na, train_en_na =DataTrainTestVal(df)
      #y_train_en, y_val_en, y_test_in_na, x_train_en, x_val_en, x_test_en_na=DataTrainTestVal(df_new)

      y_train=np.concatenate((y_train_en, y_val_en), axis=0)
      x_train=np.concatenate((x_train_en, x_val_en), axis=0)

      Train_combined = pd.concat([train_en, val_en], ignore_index=True)
      
      y_val=y_test_in_na
      x_val=x_test_en_na

      Test_Predictions=pd.DataFrame()
      Train_Predictions=pd.DataFrame()
      
      print('Data views number ============>',serial,'/',view_max,'<================')
      serial=serial+1 
      
      #view_accuracy.append(Model_Execution(models,x_train, y_train, x_val, y_val,x_test_en, y_test_in, batch_size=64))
      # test_en_na has been utilized for validation purpose of specific view
      view_accuracy_i,Test_Predictions_i,ytest_original=Model_Execution(models,x_train, y_train, x_val, y_val, x_test_en, y_test_in, test_en, scale_en, en_df, test_en_na, Train_combined, batch_size=64)

      view_accuracy.append(view_accuracy_i)
      print('shape of predictions:',np.array(Test_Predictions_i).shape)
      view_predictions.append(Test_Predictions_i)
      
    #calling compilation of the above outcome...
    Results_compile.append(MVL_results_compile(Traditional_model_pred, view_predictions, y_test_original))
    results_view_acc.append([i[0] for i in view_accuracy])


In [123]:
np.array(Results_compile)

array([[1.16890432, 1.20613193, 3.95189615, 4.12325303],
       [1.16887571, 1.1692867 , 3.95144436, 3.95844702],
       [1.16911035, 1.16901361, 3.9513877 , 3.95653254],
       [1.16927218, 1.16940378, 3.95120621, 3.95899698],
       [1.16913027, 1.16910152, 3.9515913 , 3.95770982],
       [1.16895577, 1.16939346, 3.95120295, 3.96163854],
       [1.16915733, 1.16837118, 3.95142522, 3.9557118 ],
       [1.16927057, 1.16975147, 3.95207663, 3.96233592],
       [1.16883016, 1.16860697, 3.95305443, 3.957805  ]])

In [124]:
np.array(results_view_acc)

array([list([1.3202014706151812, 1.1687170894886014]),
       list([1.1695336849115618, 1.169487180912655, 1.168995454141732]),
       list([1.1702518589884852, 1.1684754184545394, 1.168746453996697, 1.1691098398431752]),
       list([1.1697859107689788, 1.1697518663165514, 1.1690491740412632, 1.169197709952406, 1.1696677632247003]),
       list([1.1695981788817906, 1.1685860407143984, 1.1688585467680106, 1.1692001860762486, 1.1694564252857353, 1.1697883950318078]),
       list([1.1693179981959718, 1.1690580928020005, 1.1694274801477884, 1.1704332349426347, 1.1703175821831677, 1.1696554981163807, 1.1688225114720847]),
       list([1.1695655842157233, 1.1694497891102442, 1.1688523290416384, 1.1686777197067746, 1.1690482367005302, 1.1691915563016657, 1.168370708581251, 1.168054178946578]),
       list([1.1693696607420108, 1.1694358842777113, 1.1703966867571458, 1.1703292956206657, 1.1704125388854794, 1.1706011273186931, 1.1705768047009233, 1.169391089061545, 1.1686705372651471]),
       